# Entendimento de negócio

`Objetivo geral`
- Foi requisitado um modelo de performance para a identificação de clientes em _churn_
- Uma espécie de _score_ correspondente à probabilidade de um cliente entrar em _churn_
 
`Métricas`   
- Taxa atual de churn da empresa  
- Variação da taxa de churn por mês  
- Performance do modelo em classificar os clientes com churn  
- Qual o faturamento da empresa se ela impedir o churn de clientes  
- Qual o valor de um cupom de desconto daria para o cliente, a fim de evitar o churn
- Qual o custo total dessse incentivo para a empresa

# Objetivos da Sprint 1

- Descrição dos dados
- Análise descritiva dos dados
- Planejamento e substituições de NA
- Lista de hipóteses

# Descrição dos dados

Coluna | Descrição
-------|----------
RowNumber | Nº da linha
CustomerId | ID do cliente
Surname | Sobrenome do cliente
CreditScore | Pontuação de crédito do cliente para o mercado de consumo
Geography | País onde o cliente reside
Gender | Gênero do cliente
Age | Idade do cliente
Tenure| Nº de meses que o cliente permaneceu ativo
Balance  | Valor gasto pelo cliente
NumOfProducts | Nº de produtos comprados pelo cliente
HasCrCard | Indica se o cliente possui ou não um cartão de crédito
IsActiveMember | Indica se o cliente ainda tem o cadastro ativo na empresa
EstimatedSalary | Estimativa de salário mensal do cliente
Exited | Indica se o cliente está ou não em churn 

# Bibliotecas

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Carregando o dataset

In [2]:
df_raw = pd.read_csv('../data/churn.csv')

# Análise descritiva dos dados

In [3]:
# analisando colunas

df_raw.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
# checando linhas do dataset 

df_raw.shape

(10000, 14)

In [5]:
# checando colunas

df_raw.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [6]:
# checando os tipos de dados

df_raw. info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [7]:
# ver como os dados estão distribuídos

df_raw.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [8]:
# checando se há dados duplicados

df_raw.duplicated().sum()

0

In [9]:
# quantidade de clientes por sobrenome

df_raw['Surname'].value_counts()

Smith       32
Scott       29
Martin      29
Walker      28
Brown       26
            ..
Izmailov     1
Bold         1
Bonham       1
Poninski     1
Burbidge     1
Name: Surname, Length: 2932, dtype: int64

In [10]:
# quantos sobrenomes únicos existem

df_raw['Surname'].unique().shape

(2932,)

In [11]:
# quantos clientes são recém-chegados

(df_raw['Tenure'] == 0).sum()

413

In [12]:
# quantidade de clientes por produto

df_raw['NumOfProducts'].value_counts()

1    5084
2    4590
3     266
4      60
Name: NumOfProducts, dtype: int64

In [13]:
# contabilizando clientes com e sem cartão

df_raw['HasCrCard'].value_counts()

1    7055
0    2945
Name: HasCrCard, dtype: int64

In [14]:
# contabilizando clientes por gênero

df_raw['Gender'].value_counts() 

Male      5457
Female    4543
Name: Gender, dtype: int64

In [15]:
# verificando quantidade de clientes ativos

df_raw['IsActiveMember'].value_counts()

1    5151
0    4849
Name: IsActiveMember, dtype: int64

In [16]:
# verificando clientes em churn

df_raw['Exited'].value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

In [17]:
# checando países com mais clientes

df_raw['Geography'].value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

# Planejamento e substituições de NA

In [18]:
# checando dados nulos

df_raw.isna().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

Não foram verificados dados nulos.

# Levantamento de hipóteses

## Hipóteses: Clientes

 `Hipótese 1`: Mulheres estão entre os clientes mais cadastrados  
 `Hipótese 2`: Mulheres compram mais produtos do que homens  
 `Hipótese 3`: A taxa de churn de homens é maior, enquanto a de mulheres é menor  
 `Hipótese 4`: Os clientes mais ativos são adultos  
 `Hipótese 5`: Os clientes mais ativos compram mais produtos  
 `Hipótese 6`: Clientes franceses utilizam mais o cartão de crédito  
 `Hipótese 7`: Clientes franceses têm um _credit score_ maior   
 `Hipótese 8`: Clientes com salário maior compram mais  
 `Hipótese 9`: Clientes que compram mais produtos possuem menor risco de churn   
 `Hipótese 10`: Clientes com mais gastos por salário têm menos churn

## Hipóteses: Temporais

`Hipótese 11`: Clientes recém-chegados compram mais  
`Hipótese 12`: Clientes mais antigos compram menos  
`Hipótese 13`: Alguns meses possuem mais vendas/clientes ativos  
`Hipótese 14`: Clientes que não estão ativos há pelo menos 3 meses estão em churn

## Split do dataset

In [19]:
# cópia do dataset

df1 = df_raw.copy()

In [20]:
# separando features de targets

X = df1.drop('Exited', axis=1)
y = df1['Exited']

In [21]:
# separando os dados em treino, teste e validação

X_train_val, X_test, y_train_val, y_test = train_test_split(X,y, test_size=0.1, stratify=y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1, stratify=y_train_val, random_state=42)

In [22]:
# instâncias por split

print(f'Dados de treino: {X_train.shape[0]}')
print(f'Dados de validação:{X_val.shape[0]}')
print(f'Dados de teste: {X_test.shape[0]}')

Dados de treino: 8100
Dados de validação:900
Dados de teste: 1000
